# Data Exploration: Crypto Macro-Fundamental Strategy

Explore crypto prices, macro indicators, and stablecoin dynamics.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data_acquisition import DataAcquisition

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Load data
data_acq = DataAcquisition('config.yaml')
dataset = data_acq.fetch_full_dataset()
prices = dataset['prices']
events = dataset['events']

print(f"Data range: {prices.index[0]} to {prices.index[-1]}")
print(f"Total days: {len(prices)}")
print(f"Columns: {list(prices.columns)}")

## 1. Bitcoin and Ethereum Price Evolution

In [ ]:
# Normalize prices to 100
btc_normalized = prices['BTC-USD'] / prices['BTC-USD'].iloc[0] * 100
eth_normalized = prices['ETH-USD'] / prices['ETH-USD'].iloc[0] * 100

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(btc_normalized.index, btc_normalized, label='BTC-USD', linewidth=2)
ax.plot(eth_normalized.index, eth_normalized, label='ETH-USD', linewidth=2, alpha=0.7)

# Highlight crisis periods
ax.axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-04-01'), alpha=0.2, color='red', label='COVID Crash')
ax.axvspan(pd.Timestamp('2022-01-01'), pd.Timestamp('2022-12-31'), alpha=0.2, color='orange', label='Fed Tightening')
ax.axvspan(pd.Timestamp('2022-11-01'), pd.Timestamp('2022-12-01'), alpha=0.2, color='purple', label='FTX Collapse')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Normalized Price (Base=100)', fontsize=12)
ax.set_title('Bitcoin and Ethereum Price Evolution (Normalized)', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Summary statistics
btc_returns = prices['BTC-USD'].pct_change().dropna()
eth_returns = prices['ETH-USD'].pct_change().dropna()

print("\nBitcoin Summary Statistics:")
print(f"  Annualized Return: {btc_returns.mean() * 252:.2%}")
print(f"  Annualized Volatility: {btc_returns.std() * np.sqrt(252):.2%}")
print(f"  Sharpe Ratio: {btc_returns.mean() * 252 / (btc_returns.std() * np.sqrt(252)):.2f}")
print(f"  Max Drawdown: {((prices['BTC-USD'] / prices['BTC-USD'].cummax()) - 1).min():.2%}")

print("\nEthereum Summary Statistics:")
print(f"  Annualized Return: {eth_returns.mean() * 252:.2%}")
print(f"  Annualized Volatility: {eth_returns.std() * np.sqrt(252):.2%}")
print(f"  Sharpe Ratio: {eth_returns.mean() * 252 / (eth_returns.std() * np.sqrt(252)):.2f}")
print(f"  Max Drawdown: {((prices['ETH-USD'] / prices['ETH-USD'].cummax()) - 1).min():.2%}")

## 2. Traditional Finance Indicators (Treasury Yield, VIX)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# US 2-Year Treasury Yield
axes[0].plot(prices.index, prices['DGS2'], color='darkblue', linewidth=2)
axes[0].axvspan(pd.Timestamp('2022-01-01'), pd.Timestamp('2022-12-31'), alpha=0.2, color='orange')
axes[0].set_ylabel('Yield (%)', fontsize=12)
axes[0].set_title('US 2-Year Treasury Yield (DGS2)', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# VIX Index
axes[1].plot(prices.index, prices['VIX'], color='darkred', linewidth=2)
axes[1].axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-04-01'), alpha=0.2, color='red')
axes[1].axvspan(pd.Timestamp('2022-11-01'), pd.Timestamp('2022-12-01'), alpha=0.2, color='purple')
axes[1].set_ylabel('VIX Level', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_title('VIX Index (Volatility)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Correlation analysis
print("\nCorrelations with Bitcoin Returns:")
btc_returns = prices['BTC-USD'].pct_change()
dgs2_change = prices['DGS2'].diff()
vix_change = prices['VIX'].diff()

print(f"  BTC vs Δ(Treasury): {btc_returns.corr(dgs2_change):.4f}")
print(f"  BTC vs Δ(VIX): {btc_returns.corr(vix_change):.4f}")

## 3. Stablecoin Market Cap Dynamics

In [ ]:
# Plot individual stablecoins
fig, ax = plt.subplots(figsize=(14, 6))

stablecoins = ['USDT_MCap', 'USDC_MCap', 'BUSD_MCap', 'DAI_MCap']
for coin in stablecoins:
    if coin in prices.columns:
        ax.plot(prices.index, prices[coin] / 1e9, label=coin.replace('_MCap', ''), linewidth=2)

# Highlight crisis periods
ax.axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-04-01'), alpha=0.2, color='red')
ax.axvspan(pd.Timestamp('2022-11-01'), pd.Timestamp('2022-12-01'), alpha=0.2, color='purple')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Market Cap (Billions USD)', fontsize=12)
ax.set_title('Stablecoin Market Cap Evolution', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Total stablecoin market cap
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(prices.index, prices['Total_Stablecoin_MCap'] / 1e9, color='green', linewidth=2)
ax.axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-04-01'), alpha=0.2, color='red', label='COVID')
ax.axvspan(pd.Timestamp('2022-11-01'), pd.Timestamp('2022-12-01'), alpha=0.2, color='purple', label='FTX')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Market Cap (Billions USD)', fontsize=12)
ax.set_title('Total Stablecoin Market Cap', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nStablecoin Growth:")
print(f"  Peak Total MCap: ${prices['Total_Stablecoin_MCap'].max() / 1e9:.2f}B")
print(f"  Latest Total MCap: ${prices['Total_Stablecoin_MCap'].iloc[-1] / 1e9:.2f}B")

## 4. Total Crypto Market Cap

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(prices.index, prices['Total_Crypto_MCap'] / 1e12, color='purple', linewidth=2)
ax.axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-04-01'), alpha=0.2, color='red', label='COVID')
ax.axvspan(pd.Timestamp('2022-01-01'), pd.Timestamp('2022-12-31'), alpha=0.2, color='orange', label='Fed Tightening')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Market Cap (Trillions USD)', fontsize=12)
ax.set_title('Total Crypto Market Cap Evolution', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nCrypto Market Cap:")
print(f"  Peak: ${prices['Total_Crypto_MCap'].max() / 1e12:.2f}T")
print(f"  Trough (2022): ${prices[prices.index.year == 2022]['Total_Crypto_MCap'].min() / 1e12:.2f}T")
print(f"  Latest: ${prices['Total_Crypto_MCap'].iloc[-1] / 1e12:.2f}T")

## 5. Institutional Events Timeline

In [ ]:
print("Institutional Events:")
print(events.to_string())

# Plot BTC price with event markers
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(prices.index, prices['BTC-USD'], linewidth=2, label='BTC-USD')

# Mark events
for idx, row in events.iterrows():
    color = 'green' if row['impact'] > 0 else 'red'
    ax.axvline(idx, color=color, linestyle='--', alpha=0.6, linewidth=1.5)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('BTC Price (USD)', fontsize=12)
ax.set_title('Bitcoin Price with Institutional Events', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Correlation Matrix

In [ ]:
# Compute returns/changes
analysis_df = pd.DataFrame({
    'BTC_Return': prices['BTC-USD'].pct_change(),
    'ETH_Return': prices['ETH-USD'].pct_change(),
    'Treasury_Change': prices['DGS2'].diff(),
    'VIX_Change': prices['VIX'].diff(),
    'Stablecoin_Growth': prices['Total_Stablecoin_MCap'].pct_change(),
    'Crypto_MCap_Growth': prices['Total_Crypto_MCap'].pct_change()
})

# Correlation matrix
corr_matrix = analysis_df.corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Correlation Matrix: Crypto and Macro Indicators', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()